In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from keybert import KeyBERT
import yake
import string
import numpy as np
import datetime

In [2]:
sds_data = pd.read_csv('../data/sds_ds_text.csv')
#sds_non_data = pd.read_csv('../data/sds_nds_text.csv')

<IPython.core.display.Javascript object>

In [3]:
sds_data

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_year,episode_day,host_episode,class,episode_split_text,speaker
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,This is episode number one with ex-chemical en...,Kirill
1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,"Hey guys, welcome to the Podcast. I've got Rub...",Kirill
2,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,Thank you! Thanks for having me over. I'm doin...,Ruben
3,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,Awesome. It is great to hear you and for thos...,Kirill
4,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,2016-09-10,2016,Saturday,Kirill Eremenko,Database,"Yes sure. So, I'm the senior manager of analy...",Ruben
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49443,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,"Yes, right. Yes, as I mean, it actually, it ...",Jon Krohn
49444,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,Happy to.,Matar Haller
49445,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,"Nice. Well, yes, so you mentioned potentially ...",Jon Krohn
49446,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,2023-05-30,2023,Tuesday,Jon Krohn,Artificial Intelligence,Thank you for having me. This was fascinating ...,Matar Haller


In [4]:
sds_data['episode_split_text'].loc[400]

"They would come in between process and review. Because one of the key components of the review is data culling, and there are many ways you can do it. The traditional approach based on key words, they are in the past. Right now, especially if it is a large corporation, the cost is the most important factor. Because for many companies, it is a compliance issue. And if it is compliance, basically they want to do it cheap.You mentioned textual algorithms. There are different techniques which can help you isolate all non-responsive documents straight away before even starting the review. So you basically remove all these documents, and then there are certain techniques which can help you tell which data is not responsive even if you do not look at it. Basically, you pick a sample, you find the documents which are non-relevant, then you throw it back in the entire population, and then the system uses the algorithm to propagate that code, and you can tell which other documents might be rele

In [5]:
# Function to clean the text 
def lower_text(text):  
    # Lowercase the text
    if isinstance(text, float):
        return text
    text = text.lower()
    # Remove punctuation from the text
    #text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove digits from the text
    #text = ''.join(char for char in text if not char.isdigit())
    return text

In [6]:
sds_data['episode_split_text'] = sds_data['episode_split_text'].apply(lower_text)

In [8]:
sds_data.to_csv('../data/sds_BERTopic_split.csv')

## Generating keywords for Data Science Episodes

In [11]:
sds_ds_combined = pd.DataFrame(sds_data.groupby(['episode_number', 'episode_name', 'length_episode', 'class', 'guest_name', 'host_episode', 'episode_year', 'episode_date'])['episode_split_text'].agg(lambda x: ' '.join(str(i) for i in x))).reset_index()

<IPython.core.display.Javascript object>

In [12]:
sds_ds_combined

,episode_number,episode_name,length_episode,class,guest_name,host_episode,episode_year,episode_date,episode_split_text
0,1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Database,Ruben Kogel,Kirill Eremenko,2016,2016-09-10,this is episode number one with ex-chemical en...
1,2,"Machine Learning, Recommender Systems and the...",51,Machine Learning,Hadelin de Ponteves,Kirill Eremenko,2016,2016-09-14,this is session number two with machine learni...
2,3,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning,Dr. Wilson Pok,Kirill Eremenko,2016,2016-09-25,this is episode number three with nanophysics ...
3,4,"Data and Strategy, three Pillars of Research ...",60,Data Science,Brendan Hogan,Kirill Eremenko,2016,2016-10-02,this is episode four with business strategy ex...
4,5,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,Kirill Eremenko,2016,2016-10-09,this is episode number five with forensics inv...
...,...,...,...,...,...,...,...,...,...
446,679,"The A.I. and Machine Learning Landscape, with...",94,Artificial Intelligence,George Mathew,Jon Krohn,2023,2023-05-16,this is episode number 679 with george matthew...
447,680,Automating Industrial Machines with Data Scie...,30,Data Science,Allegra Alessi,Jon Krohn,2023,2023-05-19,this is episode number 680 with allegra alessi...
448,681,XGBoost,72,Machine Learning,Matt Harrison,Jon Krohn,2023,2023-05-23,"this is episode number 681 with matt harrison,..."
449,682,"Business Intelligence Tools, with Mico Yuk",28,Data Science,Mico Yuk,Jon Krohn,2023,2023-05-26,"this is episode number 682 with mico yuk, host..."


In [ ]:
# Remove episode 202 and 546 with no text in episode_split_text
#sds_processed = 
#sds_ds_processed[sds_ds_processed['episode_split_text'] == 'nan']#.reset_index(drop = True)

In [ ]:
# Function to clean the text 
def processed_text(text):  
    # Lowercase the text
    text = text.lower()
    # Remove punctuation from the text
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove digits from the text
    text = ''.join(char for char in text if not char.isdigit())
    return text

In [ ]:
sds_ds_processed['episode_split_text'].loc[1]

In [ ]:
sds_ds_processed['episode_split_text'] = sds_ds_processed['episode_split_text'].apply(processed_text)

In [ ]:
sds_ds_processed

In [ ]:
sds_ds_processed

In [ ]:
sds_ds_processed.info()

In [ ]:
sds_ds_processed['episode_split_text'].loc[400]

In [ ]:
sds_ds_processed['episode_keywords'] = None

In [13]:
sds_ds_combined['episode_keywords'] = None

In [17]:
# Function to generate 100 keywords per episode using Yake library to store keywords in one column as list
def yake_keyword_extractor(doc):
    global sds_ds_combined
    
    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n = max_ngram_size, dedupLim = deduplication_threshold , windowsSize = windowSize, top = numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)
        sds_ds_combined.at[i, 'episode_keywords'] = keywords

In [ ]:
yake_keyword_extractor(sds_ds_combined['episode_split_text'])

In [ ]:
yake_keyword_extractor(sds_ds_processed['episode_split_text'])

In [ ]:
sds_ds_processed

In [ ]:
def tuple_extractor(row):
    words = [t[0].replace(" ", "_") for t in row]
    return " ".join(words)

In [ ]:
sds_ds_processed['keywords'] = sds_ds_processed['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_ds_processed

In [ ]:
sds_ds_processed.to_csv('../data/sds_ds_keywords.csv', index = False)

## Generating keywords for non-Data Science Episodes

In [ ]:
sds_nds_processed = pd.DataFrame(sds_non_data.groupby(['episode_number', 'episode_name', 'length_episode', 'context_episode', 'guest_name', 'host_episode', 'episode_year'])['episode_split_text'].agg(lambda x: ' '.join(str(i) for i in x))).reset_index()

In [ ]:
sds_nds_processed['episode_split_text'] = sds_nds_processed['episode_split_text'].apply(processed_text)

In [ ]:
sds_nds_processed['episode_keywords'] = None

In [ ]:
# Function to generate 100 keywords per episode using Yake library to store keywords in one column as list
def yake_keyword_extractor(doc):
    global sds_nds_processed
    
    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n = max_ngram_size, dedupLim = deduplication_threshold , windowsSize = windowSize, top = numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)
        sds_nds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
yake_keyword_extractor(sds_nds_processed['episode_split_text'])

In [ ]:
sds_nds_processed['keywords'] = sds_nds_processed['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_nds_processed

In [ ]:
sds_nds_processed.to_csv('../data/sds_nds_keywords.csv', index = False)

### Count Vectorizer and Class-Based Tfidf

In [ ]:
import scipy.sparse as sp
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.preprocessing import normalize

In [ ]:
sds_ds_keywords = pd.read_csv('../data/sds_ds_keywords.csv')

keywords_per_class = sds_ds_keywords.groupby('class')['keywords'].apply(' '.join).reset_index()

# Rename the columns to match the original code
keywords_per_class = keywords_per_class.rename(columns={'class': 'Class', 'keywords': 'Document'})

In [ ]:
keywords_per_class

In [ ]:
keywords_per_class

In [ ]:
class CTFIDFVectorizer(TfidfTransformer):
    def __init__(self, *args, **kwargs):
        super(CTFIDFVectorizer, self).__init__(*args, **kwargs)

    def fit(self, X: sp.csr_matrix, n_samples: int):
        """Learn the idf vector (global term weights) """
        _, n_features = X.shape
        df = np.squeeze(np.asarray(X.sum(axis=0)))
        idf = np.log(n_samples / df)
        self._idf_diag = sp.diags(idf, offsets=0,
                                  shape=(n_features, n_features),
                                  format='csr',
                                  dtype=np.float64)
        return self

    def transform(self, X: sp.csr_matrix) -> sp.csr_matrix:
        """Transform a count-based matrix to c-TF-IDF """
        X = X * self._idf_diag
        X = normalize(X, axis=1, norm='l1', copy=False)
        return X

In [ ]:
# Create c-TF-IDF
count = CountVectorizer().fit_transform(keywords_per_class.Document)
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(keywords_per_class)).toarray()

In [ ]:
# Create bag of words using count vectorizer
count_vectorizer = CountVectorizer().fit(keywords_per_class.Document)
count = count_vectorizer.transform(keywords_per_class.Document)
words = count_vectorizer.get_feature_names_out()

In [ ]:
count_vectorizer.vocabulary_

In [ ]:
len(count_vectorizer.vocabulary_)

In [ ]:
count.shape

In [ ]:
word_counts = pd.DataFrame({
    'words': count_vectorizer.get_feature_names_out(),
    'frequency': np.array(count.sum(axis = 0)).flatten()
})
word_counts.sort_values('frequency', ascending = False)

In [ ]:
word_counts[word_counts['frequency'] < 5]

In [ ]:
# class based tfidf
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(keywords_per_class)).toarray()

In [ ]:
# find unique words in each class
unique_words_per_class = []
for i, class_docs in enumerate(keywords_per_class.Document):
    tfidf_scores = ctfidf[i]
    unique_word_indices = np.argsort(tfidf_scores)[-50:][::-1]  # index
    unique_words = [words[idx] for idx in unique_word_indices]  # feature name
    unique_words_per_class.append(unique_words)

class_unique_50 = pd.DataFrame({'Class': keywords_per_class.Class, 'unique_Words': unique_words_per_class})


In [ ]:
class_unique_50

In [ ]:
class_unique_50['unique_Words'].loc[0]

In [ ]:
class_unique_50['Class'].unique()

In [ ]:
class_unique_50.to_csv('../data/keywords_u20.csv')

### Generate word cloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:

all_keywords = ' '.join(keywords_per_class.Document)

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_keywords)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
all_keywords = ' '.join(map(str, class_unique_50.unique_Words))

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_keywords)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
sds_ds_bert = pd.read_csv('../data/sds_ds_keywords.csv')

In [ ]:
sds_ds_bert["episode_keywords"] = None
sds_ds_bert["keywords"] = None

In [ ]:
sds_ds_bert

In [ ]:
def bert_keyword_extractor(doc):
    kw_model = KeyBERT()
    global sds_ds_bert
    stopwords = list(STOP_WORDS)
    
    for i, episode_text in doc.iteritems():
        keywords = kw_model.extract_keywords(episode_text, keyphrase_ngram_range=(1, 3), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.7)
        sds_ds_bert.at[i, 'episode_keywords'] = keywords

In [ ]:
bert_keyword_extractor(sds_ds_bert['episode_split_text'][:2])

In [ ]:
sds_ds_bert['episode_keywords'].loc[0]

In [ ]:
# https://maartengr.github.io/BERTopic/index.html